# iNaturalist API Example: Finding observations with disagreeing IDs
- Link: https://jumear.github.io/stirpy/lab?path=iNat_obs_with_disagreeing_ids.ipynb
- GitHub Repo: https://github.com/jumear/stirpy

In the [iNatForum](https://forum.inaturalist.org/), folks often ask how to filter for observations with disagreements, and there is even a [Feature Request](https://forum.inaturalist.org/t/provide-a-way-to-filter-observations-by-disputed-ids/6698) to implement a basic form of this kind of filter.

Under the hood, the [Explore](https://www.inaturalist.org/observations) and [Identify](https://www.inaturalist.org/observations/identify) pages get results from the [`GET /v1/observations`](https://api.inaturalist.org/v1/docs/#!/Observations/get_observations) API endpoint. Although that endpoint provides a `identifications[i].disagreement` field in its response, which indicates whether or not an observation's identification is a disagreement, there is not a filter parameter that can be used to return only observations which have an identifications where `identifications[i].disagreement=true`.

If we move away from observations, it is possible to use [`GET /v1/identifications`](https://api.inaturalist.org/v1/docs/#!/Identifications/get_identifications) to find *identifications* where `disagreement=true`. The problem is that there are no user interfaces in the system which display identifications data from this endpoint in a human-friendly way (although there is at least one [third-party tool](https://jumear.github.io/stirfry/iNatAPIv1_identifications) that can fill this gap). However, it is possible to get the observation IDs from those identifcation records, and then display those observations by passing an `id=[comma separated list of observation IDs]` parameter to the Explore and Identify pages. So this script provides an example of how to do that in a somewhat automated way.

One limitation of this workflow is that the GET /v1/identifications endpoint provides fewer [available filter parameters](https://api.inaturalist.org/v1/docs/#!/Identifications/get_identifications) than the GET /v1/observations endpoint. For example, it is possible to filter by project when filtering for observations, but it is not possible to filter for identifications by project. However, since we are effectively going through GET /v1/observations in the end, we can apply those additional observation filter parameters on top then.

Another limitation is that identification records don't seem to have recorded `disagreement=true` prior to 2018-01-03. So if the disagreement occurred prior to then, it will not be picked up by this workflow. A similar limitation is that it is possible to withdraw or replace an initial identification where `disagreement=true` in a way that subsequent identifications will not be recorded with `disagreement=true` and so cannot be picked up by this workflow. Finding disagreements in either of these cases requires a more inefficient client-side filtering of observation records which will not be covered in this example.

Although this notebook was created with the intent of finding observations with disagreements, the basic concept of getting ids and then displaying the observations associated with those IDs can be applied to other purposes as well (ex. filtering for observations where a specific user made an identification of a specific taxon).

In [ ]:
# load required modules
from urllib.parse import parse_qs # used for parsing URL parameters
from pyodide.http import pyfetch # used for asynchronous fetching
import asyncio # used for asynchronous fetching
from copy import deepcopy # used for deep copying
import math # used for a ceiling method
#from datetime import datetime # used to convert string datetimes into actual datetimes

In [ ]:
# define the parameters needed for your request
req_params_string = 'per_page=200&disagreement=true&place_id=110679' # remember: these are filter parameters for identifications, not observations.
req_params = parse_qs(req_params_string)
req_headers_base = {'Content-Type': 'application/json', 'Accept': 'application/json'}

# to make authorized calls, set jwt to the "api_token" value from https://www.inaturalist.org/users/api_token.
# the JWT is valid for 24 hours. it can be used to do / access anything your iNat account can access. so keep it safe, and don't share it.
# you will also have to set use_authorization=True when making your API request below.
jwt = None

# define endpoints
endpoint_get_ids = {
    'method': 'GET',
    'url': 'https://api.inaturalist.org/v1/identifications',
    'max_records': 10000,
    'max_per_page': 200,
}

In [ ]:
# basic function to fetch from API and convert repsonse to JSON
async def fetchdata(url, method='GET', use_authorization=False, delay=0):
    await asyncio.sleep(delay)
    req_headers = {}
    if use_authorization and jwt:
        req_headers = deepcopy(req_headers_base)
        req_headers['Authorization'] = jwt
    #print(req_headers)
    #print(f'begin fetch: {method} {url}')
    response = await pyfetch(url, method=method, headers=req_headers)
    data = await response.json()
    print(f'fetch complete: {method} {url}')
    return data

# function to GET total_results (count) from the API
async def gettotalresults(endpoint, params={}, use_authorization=False, delay=0):
    rp = deepcopy(params)
    rp.pop('per_page', None) # remove per_page parameter, if it exists
    rp['per_page'] = ['0'] # set this to 0, since we need only the count, not the actual records
    results = await fetchdata(urlwithparams(endpoint['url'], rp), use_authorization=use_authorization, delay=delay)
    total_results = int(results['total_results'])
    print(f'total records: {str(total_results)}')
    return total_results
              
# function to GET results from the API
# if get_all_pages=True, then get all records, up to the limit that the API endpoint provides.
# query pages in parallel, with each page having a incrementally delayed start.
# (iNaturalist wants you to limit requests to ~1 req/second.)
async def getresults(endpoint, params={}, get_all_pages=False, use_authorization=False):
    results = []
    max_page = math.ceil(endpoint['max_records'] / endpoint['max_per_page']) if get_all_pages else 1
    if get_all_pages:
        # when getting all pages, make a small query first to find how many total records there are.
        # this allows us to calculate how many requests we need to make in total.
        # if total records exceeds the maximum that the API will return, then retrieve only up to the maximum.
        total_results = await gettotalresults(endpoint, params, use_authorization)
        total_pages = math.ceil(total_results / endpoint['max_per_page'])
        if total_pages < max_page:
            max_page = total_pages
        print(f'pages to retrieve: {str(max_page)}')
    async with asyncio.TaskGroup() as tg:
        tasks = []
        for i in range(max_page):
            rp = deepcopy(params)
            if get_all_pages:
                # if getting all pages, remove per_page and page parameters if they exist in the base params
                # and then set per_page = max and increment page for each request
                rp.pop('per_page', None)
                rp.pop('page', None)
                rp['per_page'] = [str(endpoint['max_per_page'])] # set this to the max if we're getting all pages
                rp['page'] = [str(i+1)]
            tasks.append(tg.create_task(fetchdata(urlwithparams(endpoint['url'], rp), use_authorization=use_authorization, delay=i)))
    for t in tasks:
        data = t.result()
        #print(data)
        results+=data['results']
    print(f'total records retrieved: {str(len(results))}')
    return results

# function to combine the base url with a set of parameters
# there's a urlencode method in urllib.parse, but it's easier to get exactly what I need using this custom code.
def urlwithparams(url_base, params={}):
    #print(params)
    url = url_base
    for p in list(params.keys()):
        #print(p)
        s = '?' if url.find('?') < 0 else '&'
        pv = ','.join(params[p])
        url += f'{s}{p}={pv}'
    # print(url)
    return url

# function to string together a list observation ids into sets of up to a max number of observations per set
# the intended use case is to create URLs linking to the iNaturalist Explore or Identification page, filtered for specific observations
def obsidstosets(obs_ids, max_set_size=500, separator=',', prefix=''):
    last_set = math.ceil(len(obs_ids) / max_set_size)
    set = 0
    obs_id_sets = []
    while set < last_set:
        set += 1
        range_from = (set - 1) * max_set_size
        range_to = set * max_set_size
        range_to = range_to if range_to < len(obs_ids) else len(obs_ids)
        obs_id_string = prefix + separator.join(map(str, obs_ids[range_from:range_to]))
        obs_id_sets.append(obs_id_string)
        print(f'Set {set}: {obs_id_string}')
    return obs_id_sets

In [ ]:
# main execution section -- part 1

# get identifications, filtered by the parameters defined in the request parameters (req_params)
ids = await getresults(endpoint_get_ids, req_params, get_all_pages=False, use_authorization=False)

In [ ]:
# main execution section -- part 2

# extract the observation ids associated with the identifications
obs_ids = [oi['observation']['id'] for oi in ids]

# string together the observation IDs, along with with a prefix, to create links to iNaturalist
# these will be printed in the cell output below. click on the URLs in the output to open a browser tab/window to that URL.
obs_id_sets = obsidstosets(obs_ids, prefix='https://www.inaturalist.org/observations/identify?id=')

In [ ]:
# optional execution section
# this can be used to fetch and accumulate additional identifications after part 1 has already run, without having to change the main request parameters
# to use the code below, set get_more_ids = True before running.

# if you order by id when you get identifications (this is the default behavior if you don't specify an order_by parameter), 
# then it should be possible to work around the max 10000 record limit of the API by using the id_above or id_below parameters.
# i purposely am not automating this process completely (because I don't want to make it too easy to accidentally get a ton of data),
# but i'm including this bit of code here to provide an idea of how to do it.
get_more_ids = False
if get_more_ids and ids:
    rp = deepcopy(req_params)
    if rp.get('order_by',['id']) == ['id']: # this only works if the records were sorted by id
        if rp.get('order',['desc']) == ['asc']:
            max_id = max([i.get('id') for i in ids])
            print(f'getting additional identifications for id_above={max_id}')
            rp.pop('id_above', None) # remove per_page parameter, if it exists
            rp['id_above'] = [str(max_id)] # set this to the max_id so that the records we get will have ids above those of the identifications we already have
        else:
            min_id = min([i.get('id') for i in ids])
            print(f'getting additional identifcations for id_below={min_id}')
            rp.pop('id_below', None) # remove per_page parameter, if it exists
            rp['id_below'] = [str(min_id)] # set this to the min_id so that the records we get will have ids below those of the identifications we already have
        ids += await getresults(endpoint_get_ids, rp, get_all_pages=False, use_authorization=False)
        print(f'identifications accumulated: {len(ids)}')